## **MSc Major Research Project**

## **Exploring the effect of daytime physical activity on sleep quality**

Melania Czobit

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing
from sklearn import metrics 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

## **Random Forest**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/MRP/FitbitClean.csv'
df = pd.read_csv(path)
df = df.drop("Unnamed: 0",axis=1)
df

,date,egoid,complypercent,meanrate,sdrate,steps,floors,sedentaryminutes,lightlyactiveminutes,fairlyactiveminutes,...,weekday,timetobed,timeoutofbed,bedtimedur,minstofallasleep,minsafterwakeup,minsasleep,minsawake,Efficiency,class
0,2015-08-01,67918.0,97.0,80.455666,14.178942,11419.0,10.0,637.0,217.0,30.0,...,5,23:53:00,9:00:00,548.0,5.0,0.0,495.0,48.0,0.911602,5
1,2015-08-02,67918.0,59.0,93.715698,10.107015,9042.0,10.0,1174.0,234.0,26.0,...,6,23:53:00,9:00:00,548.0,5.0,0.0,495.0,48.0,0.911602,5
2,2015-08-03,67918.0,97.0,77.604469,12.975431,6327.0,6.0,673.0,175.0,7.0,...,0,0:05:30,9:42:30,578.0,3.0,0.0,538.0,37.0,0.935652,5
3,2015-08-04,67918.0,59.0,90.791718,13.081469,13092.0,18.0,1118.0,205.0,35.0,...,1,0:05:30,9:42:30,578.0,3.0,0.0,538.0,37.0,0.935652,5
4,2015-08-05,67918.0,100.0,79.504982,14.706803,12689.0,47.0,563.0,261.0,15.0,...,2,23:13:00,9:00:00,588.0,2.0,0.0,547.0,39.0,0.933447,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45870,2016-07-28,36501.0,98.0,50.556137,13.650418,5965.0,6.0,647.0,183.0,26.0,...,3,23:31:30,9:26:30,596.0,0.0,0.0,538.0,58.0,0.902685,5
45871,2016-07-29,36501.0,97.0,52.905212,13.897326,8814.0,34.0,586.0,244.0,18.0,...,4,0:54:00,10:27:00,574.0,1.0,0.0,553.0,20.0,0.965096,5
45872,2016-07-30,36501.0,96.0,49.838959,8.438036,4890.0,11.0,698.0,248.0,4.0,...,5,2:15:30,10:19:30,485.0,5.0,0.0,449.0,31.0,0.935417,5
45873,2016-07-31,36501.0,93.0,52.419224,15.835029,10229.0,7.0,618.0,225.0,14.0,...,6,1:58:30,9:54:30,477.0,20.0,0.0,434.0,23.0,0.949672,5


In [ ]:
import statsmodels
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):

  results = adfuller(timeseries, autolag='AIC')
  print('Results of Augmented Dickey Fuller Test')
  print('Test Statistic: ' + str(results[0]))
  print('p-value: ' + str(results[1]))
  print('No. of lags used: ' + str(results[2]))
  print('Number of observations used: ' + str(results[3]))

  for key, value in results[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

  if results[1] <= 0.05:
    print ("Reject null hypothesis. Data is stationary.")
  else:
    print ("Fail to reject null hypothesis. Data is non-stationary.")

test_stationarity(df['Efficiency'])

Results of Augmented Dickey Fuller Test
Test Statistic: -11.59258890922617
p-value: 2.7688933530679578e-21
No. of lags used: 56
Number of observations used: 45818
Critial Values:
   1%, -3.4304927313806886
Critial Values:
   5%, -2.86160308421206
Critial Values:
   10%, -2.5668035776617875
Reject null hypothesis. Data is stationary.


In [4]:
df_RF = df.drop(['timetobed','timeoutofbed','bedtimedur','minstofallasleep','minsafterwakeup','minsasleep','minsawake'],axis=1)
df_RF

,date,egoid,complypercent,meanrate,sdrate,steps,floors,sedentaryminutes,lightlyactiveminutes,fairlyactiveminutes,veryactiveminutes,lowrangemins,fatburnmins,cardiomins,peakmins,month,weekday,Efficiency,class
0,2015-08-01,67918.0,97.0,80.455666,14.178942,11419.0,10.0,637.0,217.0,30.0,15.0,1235.0,141.0,0.0,0.0,8,5,0.911602,5
1,2015-08-02,67918.0,59.0,93.715698,10.107015,9042.0,10.0,1174.0,234.0,26.0,6.0,607.0,202.0,0.0,0.0,8,6,0.911602,5
2,2015-08-03,67918.0,97.0,77.604469,12.975431,6327.0,6.0,673.0,175.0,7.0,7.0,1317.0,71.0,0.0,0.0,8,0,0.935652,5
3,2015-08-04,67918.0,59.0,90.791718,13.081469,13092.0,18.0,1118.0,205.0,35.0,35.0,689.0,154.0,2.0,0.0,8,1,0.935652,5
4,2015-08-05,67918.0,100.0,79.504982,14.706803,12689.0,47.0,563.0,261.0,15.0,33.0,1273.0,131.0,0.0,0.0,8,2,0.933447,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45870,2016-07-28,36501.0,98.0,50.556137,13.650418,5965.0,6.0,647.0,183.0,26.0,16.0,1308.0,28.0,0.0,0.0,7,3,0.902685,5
45871,2016-07-29,36501.0,97.0,52.905212,13.897326,8814.0,34.0,586.0,244.0,18.0,18.0,1249.0,17.0,0.0,0.0,7,4,0.965096,5
45872,2016-07-30,36501.0,96.0,49.838959,8.438036,4890.0,11.0,698.0,248.0,4.0,5.0,1299.0,5.0,0.0,0.0,7,5,0.935417,5
45873,2016-07-31,36501.0,93.0,52.419224,15.835029,10229.0,7.0,618.0,225.0,14.0,50.0,1188.0,50.0,0.0,0.0,7,6,0.949672,5


In [5]:
# df_RF = df_RF.reset_index()
df_RF = df_RF.sort_values(['date', 'egoid'])

In [6]:
df_RF = df_RF.reset_index(drop=True)
df_RF.head()

,date,egoid,complypercent,meanrate,sdrate,steps,floors,sedentaryminutes,lightlyactiveminutes,fairlyactiveminutes,veryactiveminutes,lowrangemins,fatburnmins,cardiomins,peakmins,month,weekday,Efficiency,class
0,2015-08-01,11002.0,95.153846,79.716375,18.117793,14186.887363,17.392857,638.725275,249.824176,67.120879,58.013736,1105.151099,247.802198,4.189560,0.390110,8,5,0.947166,5
1,2015-08-01,11402.0,92.644258,80.514971,14.707312,11765.201681,50.683473,757.731092,213.078431,23.963585,45.697479,1176.742297,138.792717,4.691877,0.352941,8,5,0.932055,5
2,2015-08-01,14279.0,95.351955,74.965070,17.245022,9898.837989,15.092179,636.642458,240.019553,28.762570,23.561453,1206.195531,152.513966,3.611732,0.885475,8,5,0.923074,5
3,2015-08-01,14571.0,100.000000,79.816910,16.561857,15706.000000,25.000000,623.000000,218.000000,57.000000,42.000000,1221.000000,205.000000,5.000000,0.000000,8,5,0.928382,5
4,2015-08-01,14737.0,94.366477,74.216581,15.481568,13539.957386,14.017045,747.678977,244.068182,23.446023,38.519886,1253.980114,101.613636,1.241477,0.332386,8,5,0.923211,5


In [11]:
df_RF2 = df_RF.copy()
# df_RF2['Last_day_sleep'] = df_RF2.groupby(['egoid'])['Efficiency'].shift()
# df_RF2['Last_day_Diff'] = df_RF2.groupby(['egoid'])['Efficiency'].diff()
df_RF2['Next_day_sleep'] = df_RF2.groupby(['egoid'])['Efficiency'].shift(-1)
df_RF2 = df_RF2.dropna()
df_RF2

,date,egoid,complypercent,meanrate,sdrate,steps,floors,sedentaryminutes,lightlyactiveminutes,fairlyactiveminutes,veryactiveminutes,lowrangemins,fatburnmins,cardiomins,peakmins,month,weekday,Efficiency,class,Next_day_sleep
0,2015-08-01,11002.0,95.153846,79.716375,18.117793,14186.887363,17.392857,638.725275,249.824176,67.120879,58.013736,1105.151099,247.802198,4.189560,0.390110,8,5,0.947166,5,0.947166
1,2015-08-01,11402.0,92.644258,80.514971,14.707312,11765.201681,50.683473,757.731092,213.078431,23.963585,45.697479,1176.742297,138.792717,4.691877,0.352941,8,5,0.932055,5,0.932055
2,2015-08-01,14279.0,95.351955,74.965070,17.245022,9898.837989,15.092179,636.642458,240.019553,28.762570,23.561453,1206.195531,152.513966,3.611732,0.885475,8,5,0.923074,5,0.923074
3,2015-08-01,14571.0,100.000000,79.816910,16.561857,15706.000000,25.000000,623.000000,218.000000,57.000000,42.000000,1221.000000,205.000000,5.000000,0.000000,8,5,0.928382,5,0.870044
4,2015-08-01,14737.0,94.366477,74.216581,15.481568,13539.957386,14.017045,747.678977,244.068182,23.446023,38.519886,1253.980114,101.613636,1.241477,0.332386,8,5,0.923211,5,0.923211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45745,2016-07-31,98117.0,91.000000,73.599396,12.006024,1923.000000,0.000000,817.000000,118.000000,0.000000,0.000000,1282.000000,36.000000,0.000000,0.000000,7,6,0.989960,5,0.918410
45746,2016-07-31,98462.0,100.000000,72.115280,12.478164,12090.000000,7.000000,441.000000,409.000000,5.000000,4.000000,1423.000000,17.000000,0.000000,0.000000,7,6,0.915371,5,0.957187
45747,2016-07-31,98760.0,67.000000,60.901146,16.833178,11664.000000,8.000000,1102.000000,106.000000,51.000000,54.000000,918.000000,42.000000,1.000000,0.000000,7,6,0.881579,5,0.908932
45748,2016-07-31,98766.0,98.000000,69.155281,14.938157,15263.000000,21.000000,582.000000,225.000000,32.000000,55.000000,1308.000000,83.000000,0.000000,0.000000,7,6,0.968460,5,0.929062


### **Regression**

In [ ]:
X = df_RF2.drop(['Efficiency','class', 'Next_day_sleep'], axis=1) 
y = df_RF2['Next_day_sleep']

tscv = TimeSeriesSplit(n_splits=5)

rmse=[]
mae = []
r2 = []

for train_index, test_index in tscv.split(X):
  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  min_max_scaler = preprocessing.MinMaxScaler()
  X_train = min_max_scaler.fit_transform(X_train)
  X_test = min_max_scaler.fit_transform(X_test)

  mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
  # mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=5)
  mdl.fit(X_train, y_train)
  y_pred = mdl.predict(X_test)
  RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  rmse.append(round(RMSE,3))
  MAE = mean_absolute_error(y_test, y_pred)
  mae.append(round(MAE,3))
  R2 = r2_score(y_test, y_pred)
  r2.append(round(R2,3))

dfResult = pd.DataFrame()
dfResult['Fold'] = [0,1,2,3,4]
dfResult['Model'] = ['Random Forest','Random Forest','Random Forest','Random Forest','Random Forest']
dfResult['RMSE'] = rmse
dfResult['MAE'] = mae
dfResult['R2'] = r2

print('Average RMSE:', (sum(rmse)/len(rmse)))
print('Average MAE:', (sum(mae)/len(mae)))
print('Average R2:', (sum(r2)/len(r2)))

Average RMSE: 0.0506
Average MAE: 0.0308
Average R2: 0.13479999999999998


In [ ]:
dfResult.set_index('Fold')

,Model,RMSE,MAE,R2
Fold,,,,
0,Random Forest,0.050,0.032,0.005
1,Random Forest,0.054,0.035,-0.198
2,Random Forest,0.046,0.029,0.247
3,Random Forest,0.039,0.026,0.429
4,Random Forest,0.064,0.032,0.191


In [14]:
X = df_RF2.drop(['Efficiency','class', 'Next_day_sleep'], axis=1) 
y = df_RF2['Next_day_sleep']

tscv = TimeSeriesSplit(n_splits=5)

rmse=[]
mae = []
mape = []
r2 = []

for train_index, test_index in tscv.split(X):
  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  min_max_scaler = preprocessing.MinMaxScaler()
  X_train = min_max_scaler.fit_transform(X_train)
  X_test = min_max_scaler.fit_transform(X_test)

  # mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
  mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=5)
  mdl.fit(X_train, y_train)
  y_pred = mdl.predict(X_test)
  RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  rmse.append(round(RMSE,3))
  MAE = mean_absolute_error(y_test, y_pred)
  mae.append(round(MAE,3))
  mape_value = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
  mape.append(mape_value)
  R2 = r2_score(y_test, y_pred)
  r2.append(round(R2,3))

dfResult2 = pd.DataFrame()
dfResult2['Fold'] = [0,1,2,3,4]
dfResult2['Model'] = ['Random Forest','Random Forest','Random Forest','Random Forest','Random Forest']
dfResult2['RMSE'] = rmse
dfResult2['MAE'] = mae
dfResult2['R2'] = r2

print('Average RMSE:', (sum(rmse)/len(rmse)))
print('Average MAE:', (sum(mae)/len(mae)))
print('Average MAPE:', (sum(mape)/len(mape)))
print('Average R2:', (sum(r2)/len(r2)))

Average RMSE: 0.0504
Average MAE: 0.031
Average MAPE: 3.6475145075114668
Average R2: 0.1472


In [15]:
dfResult2.set_index('Fold')

,Model,RMSE,MAE,R2
Fold,,,,
0,Random Forest,0.054,0.033,-0.136
1,Random Forest,0.050,0.033,-0.034
2,Random Forest,0.046,0.029,0.266
3,Random Forest,0.039,0.028,0.423
4,Random Forest,0.063,0.032,0.217


In [17]:
df_RF2['Last_day_sleep'] = df_RF2.groupby(['egoid'])['Efficiency'].shift()
df_RF2['Last_day_Diff'] = df_RF2.groupby(['egoid'])['Efficiency'].diff()
df_RF2 = df_RF2.dropna()

X = df_RF2.drop(['Efficiency','class', 'Next_day_sleep'], axis=1) 
y = df_RF2['Next_day_sleep']

tscv = TimeSeriesSplit(n_splits=5)

rmse=[]
mae = []
mape = []
r2 = []

for train_index, test_index in tscv.split(X):
  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  min_max_scaler = preprocessing.MinMaxScaler()
  X_train = min_max_scaler.fit_transform(X_train)
  X_test = min_max_scaler.fit_transform(X_test)

  # mdl = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=0)
  mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=5)
  mdl.fit(X_train, y_train)
  y_pred = mdl.predict(X_test)
  RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  rmse.append(round(RMSE,3))
  MAE = mean_absolute_error(y_test, y_pred)
  mae.append(round(MAE,3))
  mape_value = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
  mape.append(mape_value)
  R2 = r2_score(y_test, y_pred)
  r2.append(round(R2,3))

dfResult3 = pd.DataFrame()
dfResult3['Fold'] = [0,1,2,3,4]
dfResult3['Model'] = ['Random Forest','Random Forest','Random Forest','Random Forest','Random Forest']
dfResult3['RMSE'] = rmse
dfResult3['MAE'] = mae
dfResult3['R2'] = r2

print('Average RMSE:', (sum(rmse)/len(rmse)))
print('Average MAE:', (sum(mae)/len(mae)))
print('Average MAPE:', (sum(mape)/len(mape)))
print('Average R2:', (sum(r2)/len(r2)))

Average RMSE: 0.038
Average MAE: 0.0256
Average MAPE: 2.9589576263067374
Average R2: 0.5194000000000001


In [18]:
dfResult3.set_index('Fold')

,Model,RMSE,MAE,R2
Fold,,,,
0,Random Forest,0.036,0.026,0.509
1,Random Forest,0.035,0.027,0.502
2,Random Forest,0.034,0.024,0.587
3,Random Forest,0.040,0.026,0.400
4,Random Forest,0.045,0.025,0.599


### **Classification**

In [ ]:
df_RF2['class'].value_counts()

5    45178
3      336
4      179
2       52
1        5
Name: class, dtype: int64

In [ ]:
X = df_RF2.drop(['Efficiency','class'], axis=1) 
y = df_RF2['class']

tscv = TimeSeriesSplit(n_splits=5)

acc=[]

for train_index, test_index in tscv.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  score =accuracy_score(y_test,y_pred)
  acc.append(score)

print('Average accuracy:', round((sum(acc)/len(acc)),4))

Average accuracy: 0.9946


In [ ]:
df_RF3 = df_RF2.drop(['complypercent', 'meanrate', 'sdrate', 'steps', 'floors', 'lightlyactiveminutes', 'fairlyactiveminutes','veryactiveminutes','fatburnmins','cardiomins','peakmins'], axis=1)
df_RF3 = df_RF3.drop(['month','weekday','Efficiency'], axis=1)
X = df_RF3.drop(['class'], axis=1) 
y = df_RF3['class']
tscv = TimeSeriesSplit(n_splits=5)

acc=[]

for train_index, test_index in tscv.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  min_max_scaler = preprocessing.MinMaxScaler()
  X_train = min_max_scaler.fit_transform(X_train)
  X_test = min_max_scaler.fit_transform(X_test)

  clf = RandomForestClassifier(n_estimators=1000, class_weight='balanced')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  score =accuracy_score(y_test,y_pred)
  acc.append(score)

print('Average accuracy:', round((sum(acc)/len(acc)),4))

Average accuracy: 0.992


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier
# model = BalancedRandomForestClassifier(n_estimators=10)

X = df_RF2.drop(['Efficiency','class'], axis=1) 
y = df_RF2['class']

tscv = TimeSeriesSplit(n_splits=5)

acc=[]
p = []
r = []
f1 = []

for train_index, test_index in tscv.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]      
  y_train, y_test = y.iloc[train_index],  y.iloc[test_index]

  X_train = X_train.drop(['date'],axis=1)
  X_test = X_test.drop(['date'], axis=1)

  clf = BalancedRandomForestClassifier(n_estimators=1000, max_depth=10)
  # clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  score =accuracy_score(y_test,y_pred)
  acc.append(score)
  precision = metrics.precision_score(y_test,y_pred, average='macro')
  p.append(precision)
  recall = metrics.recall_score(y_test, y_pred, average='macro')
  r.append(recall)
  f1score = metrics.f1_score(y_test,y_pred, average='macro')
  f1.append(f1score)
print('Average accuracy:', round((sum(acc)/len(acc)),4))
print('Average precision:', round((sum(p)/len(p)),4))
print('Average recall:', round((sum(r)/len(r)),4))
print('Average F1-Score:', round((sum(f1)/len(f1)),4))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Average accuracy: 0.8678
Average precision: 0.428
Average recall: 0.5076
Average F1-Score: 0.4261
